# HTTP session management
Server assigns session id to new users in response header and creates a cookie for session:
```
Set-Cookie: sessionid=<cookie-value>
```

Cookies are sent back to server in every subsequent request header;
```
Cookie: sessionid=<cookie-value>; key=value;
```

Session sniffing - waiting for user login to use his session id.




## Cookie syntax
- `Secure` cookie - will only be sent to server via HTTPS
- Cookie will be rejected by client if `Domain` is different from server's.
- `Path` indicates the path that must exist in the requested URL for the browser to send the Cookie header
- Browser will expire cookie after number of seconds has passed (`Max-Age`) or at specific date (`Expires`). Those are _permanent cookies_
- Cookies can be deleted when session ends/browser is closed - when `Max-Age` is not set. Those are _session cookies_. 
- `HttpOnly` forbids JavaScript from accessing the cookie.
- `SameSite`: None, Lax, Strict - defines if broser should send back the cookie for third-party requests
  - If the user is on www.web.dev and requests an image from static.web.dev then that is a same-site request.
  - If the user is on your-project.github.io and requests an image from my-project.github.io that's a cross-site request
  - Strict - Domain in URL bar equals the cookie’s domain (first-party) AND the link isn’t coming from a third-party
  - Lax - Domain in URL bar equals the cookie’s domain (first-party)
  - None - No domain limitations and third-party cookies can fire


```
Set-Cookie: <cookie-name>=<cookie-value>
Set-Cookie: <cookie-name>=<cookie-value>; Expires=<date>
Set-Cookie: <cookie-name>=<cookie-value>; Max-Age=<number>
Set-Cookie: <cookie-name>=<cookie-value>; Domain=<domain-value>
Set-Cookie: <cookie-name>=<cookie-value>; Path=<path-value>
Set-Cookie: <cookie-name>=<cookie-value>; Secure
Set-Cookie: <cookie-name>=<cookie-value>; HttpOnly

Set-Cookie: <cookie-name>=<cookie-value>; SameSite=Strict
Set-Cookie: <cookie-name>=<cookie-value>; SameSite=Lax
Set-Cookie: <cookie-name>=<cookie-value>; SameSite=None; Secure

// Multiple attributes are also possible, for example:
Set-Cookie: <cookie-name>=<cookie-value>; Domain=<domain-value>; Secure; HttpOnly
```

Cookies are used for:
- session management
- personalization
- tracking

## Django session
To manually set cookies use HttpResponse:
```
from django.http import HttpResponse

response = HttpResponse()
response.set_cookie(
  'cookie-name',
  'cookie-value',
  secure=True,
  domain='alice.com',
  max_age=42, )
```

To retrieve session info use `session` object from request:
```
request.session['name'] = 'Alice'
name = request.session.get('name', 'Bob')
request.session['name'] = 'Charlie'
del request.session['name']
```



Django uses `SESSION_SERIALIZER` -> `JSONSerializer` or `PickleSerializer`.

Django persists sessions using selected `SESSION_ENGINE` eg. Database-based sessions.

Django uses the `SECRET_KEY` setting for keyed hashing, not encryption


# User authentication

Use `django-registration` extension for account registration workflow

Use default Django CBV's for handling login (eg. `LoginView`), logout and registration

For requiring login use decorator for FBV and mixin for CBV
```
@login_required
LoginRequiredMixin
```

For testing user authentication you can use
```
from django.contrib.auth import get_user_model
from django.test import TestCase
```

# User password management

Use built-in password change views eg. `PasswordChangeView`

Use `AUTH_PASSWORD_VALIDATORS` to check password validity

## Custom validator


In [ ]:
from django.core.exceptions import ValidationError
from django.utils.translation import gettext_lazy as _

class PassphraseValidator:

    def __init__(self, dictionary_file='/usr/share/dict/words'):
        self.min_words = 4
        with open(dictionary_file) as f:
            self.words = set(word.strip() for word in f)

    def get_help_text(self):
        return _('Your password must contain %s words' % self.min_words)
    
    def validate(self, password, user=None):
        tokens = password.split(' ')
        
        if len(tokens) < self.min_words:
            too_short = _('This password needs %s words' % self.min_words)
            raise ValidationError(too_short, code='too_short')

        if not all(token in self.words for token in tokens):
            not_passphrase = _('This password is not a passphrase')
            raise ValidationError(not_passphrase, code='not_passphrase')

# Authorization

## Application-level authorization
Django represents permissions with a built-in model known as Permission. The Permission model is the most atomic element of Django authorization. Each user can be associated with zero to many permissions. Permissions fall into two categories:
- Default permissions, created automatically by Django
- Custom permissions, created by you

Django automatically creates four default permissions for each new model. These permissions are created behind the scenes when you run migrations. These permissions allow a user to create, read, update, and delete a model.

Custom permissions can be added in `Meta` class of a model.

Like default permissions, custom permissions are created automatically during migrations.

In [ ]:
class AuthenticatedMessage(Model):
    message = CharField(max_length=100)
    mac = CharField(max_length=64)

    class Meta:
        permissions = [
            ('send_authenticatedmessage', 'Can send msgs'),
            ('receive_authenticatedmessage', 'Can receive msgs'),
        ]

# OAuth 2